In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import glob
import os
from IPython.display import display
import matplotlib.pyplot as plt
import json
from sklearn import preprocessing
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
from keras.preprocessing.text import Tokenizer
import tensorflow_addons as tfa
import decimal
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
import math

import warnings
warnings.filterwarnings('ignore')

In [3]:
trailers_csv = "/Volumes/Seagate/natasha-diploma/trailers.csv"
trailers_df = pd.read_csv(trailers_csv, index_col=None, header=0)
trailers_df.drop(trailers_df.columns[trailers_df.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
display(trailers_df.head())

,title_x,duration,release_date,restrict,orig_title,kinopoisk_id,imdb_rating,kinopoisk_rating,synopsis,categories,...,Theme_x,Place of act_y,Qualities_y,Time of act_y,Based on_y,Audience_y,Mood_y,Subgenre_y,About_y,Theme_y
0,Философия Фила,5816.0,2019-04-05,18.0,Phil,960568,5.4,5.86,"Фил – стоматолог, страдающий от депрессии. Его...",{Фильмы},...,"['развлекательные', 'психологический']",['США'],['дебютный'],"['21-й век', 'новейшее время']",['оригинального сценария'],"['для молодёжи', 'для взрослых']","['грустный', 'депрессивный', 'трогательный']","['Драмы', 'Комедийные']","['преследования', 'самозванцы', 'расследование...","['развлекательные', 'психологический']"
1,Зубная фея,5123.0,2019-04-02,18.0,Tooth Fairy,1126022,2.2,0.00,Старая женщина начинает рассказывать легенду о...,{Фильмы},...,['развлекательные'],['не определено'],"['зрелищные', 'жанровый', 'неожиданный финал']",['не определено'],"['эпоса и легенд', 'оригинального сценария']","['для взрослых', 'для мужчин', 'для подготовле...","['трагичные', 'мрачные', 'шокирующие', 'захват...","['ужасы', 'триллер']","['преследования', 'страхи', 'дети', 'молодежь'...",['развлекательные']
2,Проснись,4667.0,2019-01-19,16.0,Wake Up,1069713,2.9,0.00,"Врач-психиатр исследует дневник девушки, котор...",{Фильмы},...,"['психологический', 'развлекательные']",['не определено'],"['жанровый', 'дебютный']",['новейшее время'],['оригинального сценария'],"['для взрослых', 'для подготовленного зрителя'...","['захватывающие', 'жестокие', 'напряженные', '...","['Драмы', 'Боевики', 'триллер']","['психотерапевты', 'кровь', 'тайна смерти', 'п...","['психологический', 'развлекательные']"
3,[4k] Адская кухня,6148.0,2019-08-09,16.0,The Kitchen,1117988,5.5,5.50,"Смотрите в 4К! Три жены ирландских гангстеров,...",{Фильмы},...,"['авантюрные', 'гангстерская']","['большой город', 'США']","['жанровый', 'остросюжетные']",['20-й век'],['комиксов'],"['для мужчин', 'для женщин', 'для взрослых']","['захватывающие', 'авантюрный', 'жестокие', 'н...","['Боевики', 'Драмы', 'криминальные']","['враги или вражда', 'женщины', 'антигерои', '...","['авантюрные', 'гангстерская']"
4,[4k] Покемон. Детектив Пикачу,6013.0,2019-05-02,12.0,Pokémon Detective Pikachu,994864,6.6,6.56,Смотрите в 4K! Элитный детектив Пикачу отправл...,"{Для детей,Фильмы}",...,"['развлекательные', 'образовательный']","['вымышленная страна (город)', 'большой город']","['частично анимационный', 'зрелищные', 'кассов...",['21-й век'],"['оригинального сценария', 'сериалов']","['для детей 13-16 лет', '9 лет', '8 лет', '7-1...","['авантюрный', 'захватывающие', 'динамичный', ...","['детские', 'Приключенческие', 'Фантастические...","['притяжение противоположностей', 'риск', 'пре...","['развлекательные', 'образовательный']"


In [4]:
sessions_csv = "/Volumes/Seagate/natasha-diploma/sessions.csv"
sessions_df = pd.read_csv(sessions_csv, index_col=None, header=0)
sessions_df.drop(sessions_df.columns[sessions_df.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
display(sessions_df)

,0,1,2,3,4,5,6,7,8,9
0,403a42b604f504069c41d39c5a4fc2,23ee5640ec588ffaa78afa3268b04a,5b150a1cae009459493bd0fe5c7b3e,e291f8ae1de4a38c71027e9c91d51c,34cfd44ad5cc2d63441ebf437eccb5,02fbcf93a730c9c9d5e65aef41c67e,26b3f811a6b9ed75d3cc5198105fa4,6fcff414343456465369b05f6851eb,e2f15819192cded77ed1825a129c0c,41b772a5b18bc724562bfb60f4f3ac
1,a5b2dd3ed41aabb5733c7cfffff0bc,25e573b81ab69d6cf700a51281fc2a,49c58f679f72e0f02b97bdeee1a1bb,c0be492c14bcec5ab3dfd34d95d6f5,b2274b9b4c8c4dc2f79d417ddd71e5,7fd08607f52b89a753437eee177e32,25d41576f3529994c878a5d15f6dc7,ece16e15b2286f8287e8935f12b6c2,806c0ed32e135afb8db4f3aad71acf,f10b4adf1c732b6b260791fb24f430
2,80870ec50515107ba7034ab758cebe,cd522afebcf91625c3cd811f3fb6e2,061a59839837a123a916a70947f075,07e873c5aac8ffaf97897fbd0dfbf6,60fb638ff7ab469be8637111fdc837,d49c4a9b748b9c7abb9d98cdecebef,b0b062dc25c4dada316183941b100c,c7f809aab571e9acdf799a78c5ef61,67c38de267396951227897efc503e1,7b457a5d30f3cb01b6998390ec7abf
3,d66440e7524c0a8b9b206fbb7295ba,046f93c91d0befd4625f00e07eb473,199cf6d89a2f06fc552ebf86105d7d,2adfc14bbc0fba5b5d616d6fcfe3de,a458a5a362af9a7774780eb4e25365,88bbc5912f946b1fb77fdeaa3f5d4a,6d0bb6959972ac92c10d7445982a94,274869b912b256df69b9e7c5c0cb11,2c5bf20534b80ee47bed9a90b405a0,f456635bb3d2567a3a61133fa9dd89
4,231b80c884b2f1390e17f09a187889,bd6707bb04cfb8880a5f783c85be20,bbd5df907c099d2cc2b5285b08038e,514f3ee1f00df05dc3900251972245,961c336b0c2fa04c883c639dfee1ba,d6159553e41857fe23df9d3e746284,d2cd45a49d53702df403cffddd53e6,c0314a446fbef65ce9626b037a70da,d6e8f6d21631149a90ac940d3ee227,e762f208419b7449c0731037262bb3
...,...,...,...,...,...,...,...,...,...,...
196,91e8db1b91acc7bf903bf670461db3,f65c8906c07b627a760a0671de7144,26ba677f7ff11ac529f5f69c54cb0b,be6360babdb2e5910c5bc6f4f04f0d,705c23ed5ae1bc8824fc955b82094b,74733260a8a5143b2eb20c4a62940a,afed0e34e5af6958fef610a0ac5565,0a49f9b4576a63be41cb96908eba5e,baec94926106f54090568b653e84b4,8a2b5f96cc30660b28636fbe2664d5
197,c718bcdf6240802ba7123a7a631199,5e62a64d4f0d6ca3fce397267bf9f3,c5d40f1adeb446df77e65815da405d,bbf4b4a3db602ab9b88845114f7244,0f462e6bbd719e012902d8814d0e35,e5bca888fec7f8c0116db0002a7077,8a2483e11828332a9e2fc8659ae38f,bb59c82aeb5e44eb5735d5a1c016b8,8d3877a1b26b10642d125ec4fc4c9b,ae90166aa81ab2bf878dace61a5620
198,ee78a3aae1a69f2e4b82f26d41d594,11fe2dd4cccaf0567c878e19e1811a,aab18e84a97780bafb188fd33cb2a9,838cb9d91dd080147bcc18982bcdbb,972821d03656142f33707487f51b66,d6d45793f5b093e8bdf3d8ae4952b6,28731720ee3330b7107cec47168b8c,dad45a9302ad916f6901b86a744cdd,644bf6d0bdf6c9cddf71f5ab4cf7d4,4774caaf01df3d5d982e0cb77b1a13
199,b0b062dc25c4dada316183941b100c,80870ec50515107ba7034ab758cebe,d49c4a9b748b9c7abb9d98cdecebef,67c38de267396951227897efc503e1,7b457a5d30f3cb01b6998390ec7abf,67f32aff5488778cb4bbb6b21acc7d,3d558c3e37ccb70937b171aeb67bbb,97474799458a67e30a70f66407420b,0dbe493275a25285829b979316d566,ba54f9d576e93f2b0cf4622a92310f


In [5]:
movies_id = sessions_df.to_numpy()
movies_id = np.array(movies_id).flatten()
vocab = np.unique(movies_id)
vocab_len = len(vocab)

print(vocab)
print(vocab_len)
print(movies_id.shape)

['000b366b20d0f41187f6ae895e5d7a' '001db9af2dcc57d45d653aeb33171e'
 '00d3113bae64107671f55b51e183b9' '016254be26e2789ef3253e95ded1ea'
 '019fa155167f13b7be3444c46810ce' '01ab32976170c8eebe4054fd66cc39'
 '02817e267c3e407bc7b207f6ed9f34' '02a7f8e9b88ca2a74c6637332b0998'
 '02fbcf93a730c9c9d5e65aef41c67e' '046f93c91d0befd4625f00e07eb473'
 '05c92546aa14ea6148088b7c713be4' '06030fb2a64f0fc9df81ff76eea9d5'
 '06132809afbe1c781e1b9605452081' '061a59839837a123a916a70947f075'
 '06cd00fa7fce77cc7c1ad0a64d163e' '06d4e0812e40c50f93b3407878e744'
 '06f71f27fd9318f8b0f4f4dd69e522' '0713394356c9d9daf29edc39231352'
 '077374b7238b28b0a4aefa135f2f45' '07e873c5aac8ffaf97897fbd0dfbf6'
 '08712d35cf70496630e16b24dae005' '09bf18e7baee13e303f4b77aeaadbc'
 '0a091e4ffb26e3fc15d37939cd5682' '0a49f9b4576a63be41cb96908eba5e'
 '0b5aaa6d4b9941b2f4a31f9480d757' '0bdcec51b6924793d1fbd815daaa51'
 '0c691e3e6b52752ab6717b3729e326' '0dbe493275a25285829b979316d566'
 '0e0c51266d47f2c02625e16f61fa50' '0f462e6bbd719e012902d8814d0

In [6]:
def getVideoInfo(info_df):
    return [
        info_df['temperature'].to_list(),
        info_df['brightness'].to_list(),
        info_df['colorfulness'].to_list(),
        info_df['energy'].to_list(),
        info_df['tempo'].to_list(),
        info_df['amplitude'].to_list(),
        info_df['spectral_rolloff'].to_list(),
        info_df['mfcc'].to_list()
    ]


In [7]:
info = {}
scenes_array = []
del_array_name = []
info_folder = "/Volumes/Seagate/natasha-diploma/videoinfo"
for i in range(0, len(trailers_df)):
    name = trailers_df.iloc[i]['trailers_name']
    try:
        info_csv = info_folder + '/' + name + '.csv'
        info_df =  pd.read_csv(info_csv, index_col=None, header=0)
        info[name] = getVideoInfo(info_df.iloc[: , 4:])
        scenes_array.append(info[name])
    except:
        del_array_name.append(name)


In [8]:
MAX_SCENES = 396

In [9]:
scenes_data = []
for i in range(0, len(movies_id)):
    scenes_data.append(info[movies_id[i]])

In [12]:
def pad_nested_arrays(seq, maxlen_sent):
    new_seq = []
    for i in range(len(seq)):
        size = len(seq[i])
        if (maxlen_sent == size):
            new_seq.append(np.pad(seq[i], 0, 'mean'))
        else:
            size_beg = (maxlen_sent - size) // 2
            size_end = size_beg + 1
            new_seq.append(np.pad(seq[i], (size_beg, size_end), 'mean')[:396])
            
    new_rounded = []
    for i in range(len(new_seq)):
        k_scenes = []
        for k in range(len(new_seq[i])):
            if new_seq[i][k] == float('inf'):
                k_scenes.append(float(0))
            else:
                if new_seq[i][k] < 10000:
                    k_scenes.append(float("{:.2f}".format(new_seq[i][k])))
                else:
                    k_scenes.append(float(10000))
        new_rounded.append(k_scenes)
        
    return new_rounded

padded = np.stack([pad_nested_arrays(r, MAX_SCENES) for r in scenes_data], axis=0)

In [13]:
scaler = MinMaxScaler((0, 1))

normalized = []
for i in range(len(padded)):
    normalized.append(scaler.fit_transform(np.nan_to_num(padded[i])))

In [14]:
dataset = tf.data.Dataset.from_tensor_slices(normalized[:1024])
test_dataset = tf.data.Dataset.from_tensor_slices(normalized[1024:])

In [15]:
seq_length = 10
examples_per_epoch = len(normalized[:1024]) // (seq_length + 1)

In [16]:
sequences = dataset.batch(seq_length + 1, drop_remainder=True)

In [17]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [18]:
for input_example, target_example in dataset.take(1):
    print("Input :", input_example)
    print("Target:", target_example)

Input : tf.Tensor(
[[[1.         1.         1.         ... 1.         1.         1.        ]
  [0.00565002 0.00565002 0.00565002 ... 0.00565002 0.00565002 0.00565002]
  [0.00569789 0.00569789 0.00569789 ... 0.00569789 0.00569789 0.00569789]
  ...
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.41138542 0.41138542 0.41138542 ... 0.41138542 0.41138542 0.41138542]
  [0.00136638 0.00136638 0.00136638 ... 0.00136638 0.00136638 0.00136638]]

 [[0.00527064 0.00527064 0.00527064 ... 0.00527064 0.00527064 0.00527064]
  [0.01516704 0.01516704 0.01516704 ... 0.01516704 0.01516704 0.01516704]
  [0.01403382 0.01403382 0.01403382 ... 0.01403382 0.01403382 0.01403382]
  ...
  [0.         0.         0.         ... 0.         0.         0.        ]
  [1.         1.         1.         ... 1.         1.         1.        ]
  [0.00475579 0.00475579 0.00475579 ... 0.00475579 0.00475579 0.00475579]]

 [[1.         1.         1.         ... 1.         1.         1.        ]
  [

In [1]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (dataset
    .shuffle(BUFFER_SIZE)
#     .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

NameError: name 'dataset' is not defined

In [21]:
MAX_SEQUENCE_NUM = vocab_len #366
BATCH = 64
SEQUENCE_LEN = 10
FEATURES_LEN = 8
tf.config.run_functions_eagerly(True)

In [ ]:
class ScenesModel(tf.keras.Model):
    def __init__(self):
        super(ScenesModel, self).__init__()
        
        self.scenes_lstm = tf.keras.layers.LSTM(MAX_SCENES, return_sequences=True)
        self.scenes_dense = tf.keras.layers.Dense(1, activation='relu', name='scenes_dense')
        
        
    def call(self, inputs):
        lstm_res = self.scenes_lstm(inputs)
        output = self.scenes_dense(lstm_res)
        
        return output
    
    
class TestModel(tf.keras.Model):
    def __init__(self, scenes_model):
        super(TestModel, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(10, 1)
        self.seq_encoder = tf.keras.layers.TimeDistributed(scenes_model, name='seq_linking')
        self.seq_lstm = tf.keras.layers.LSTM(SEQUENCE_LEN, return_sequences=True)
        self.dense = tf.keras.layers.Dense(MAX_SEQUENCE_NUM, activation='softmax', name='output')
        
    def call(self, inputs):
        inputs = self.embedding(inputs)
        inputs = tf.transpose(inputs, perm=[0, 2, 1, 3]) # _, scenes, features
        trailers_encoder_out = self.seq_encoder(inputs)
        encoder_squeezed_out = tf.squeeze(inputs, [3])
        lstm_res = self.seq_lstm(encoder_squeezed_out)
        output = self.dense(lstm_res)
        
        return output
    

scenes_model = ScenesModel()
seq_model = TestModel(scenes_model)

In [36]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = seq_model(input_example_batch)

In [38]:
seq_model.summary()
loss = tf.losses.CategoricalCrossentropy(from_logits=True)


Model: "test_model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      multiple                  10        
_________________________________________________________________
seq_linking (TimeDistributed multiple                  630829    
_________________________________________________________________
lstm_15 (LSTM)               multiple                  760       
_________________________________________________________________
output (Dense)               multiple                  4026      
Total params: 635,625
Trainable params: 635,625
Non-trainable params: 0
_________________________________________________________________


In [39]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

ValueError: Shapes (10, 8, 396) and (10, 396, 366) are incompatible

In [40]:
seq_model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

In [41]:
EPOCHS = 10
history = seq_model.fit(dataset, epochs=EPOCHS)

Epoch 1/10


ValueError: Shapes (10, 8, 396) and (10, 396, 366) are incompatible

In [ ]:
test = normalized[1024:]

test_values = []
acc = 0
array = tokenized[1024:]
_all = 0
for i in range(len(array)):
    if (len(test_values) == 10):
        predicted = model.predict(test_values)
        sampled_indices = tf.random.categorical(predicted[0], num_samples=1)
        sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
        if (array[i] == sampled_indices):
            acc += 1
        _all += 1
        test_values = []
        break
    test_values.append(array[i])


In [ ]:
predicted = model.predict()
sampled_indices = tf.random.categorical(predicted[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()